In [1]:
import logging
from dataclasses import dataclass, asdict
from typing import cast, Any

logging.basicConfig(level=logging.DEBUG)

class SkuValidator:
    """Validates that a string is an SKU.
    
    A valid SKU must have a dash in the middle but notat either end.
    """
    
    def __set_name__(self, owner: Any, name: str) -> None:
        """Store private an public name.
        
        :param Any owner: owner class using this descriptor in one of its fields.
        :param str name: Name of attribute in owner instance.
        """
        self.name = f"_{name}"
        self.public_name = name

    def __get__(self, obj: Any, objtype=None) -> str:
        """Attribute access method.
        
        :param Any obj: Instance of attribute being accessed.
        :param Any objtype: Class of instance being accessed.
        :return: Value stored in private attribute.
        """
        return getattr(obj, self.name)
    
    def __set__(self, obj: Any, value: str) -> None:
        """Attribute assign method.
        
        :param Any obj: Instance of attribute being assigned.
        :param str value: Value being assigned to attribute.
        """
        validated_data = self.validate(value)
        setattr(obj, self.name, validated_data)

    def validate(self, value: str) -> str:
        """Validate SKU format."""
        if any([
            "-" not in value,
            value.startswith("-"),
            value.endswith("-")]):
            raise AttributeError(
                f"Field '{self.public_name}' must contain a dash ('-') "
                "in the middle of the string.")
        return value
            

class PositiveIntValidator:
    """Validates an integer is positive."""
    def __set_name__(self, owner, name):
        """Store private an public name.
        
        :param Any owner: owner class using this descriptor in one of its fields.
        :param str name: Name of attribute in owner instance.
        """
        self.name = f"_{name}"
        self.public_name = name

    def __get__(self, obj, objtype=None):
        """Attribute access method.
        
        :param Any obj: Instance of attribute being accessed.
        :param Any objtype: Class of instance being accessed.
        :return: Value stored in private attribute.
        """
        return getattr(obj, self.name)
    
    def __set__(self, obj, value):
        """Attribute assign method.
        
        :param Any obj: Instance of attribute being assigned.
        :param str value: Value being assigned to attribute.
        """
        validated_data = self.validate(value)
        setattr(obj, self.name, validated_data)
        
    def validate(self, value: int) -> int:
        """Validate positive int."""
        if value < 0:
            raise AttributeError(f"{self.public_name} must be positive")
        return value


@dataclass
class Item:
    sku: str = cast(str, SkuValidator())
    quantity: int = cast(int, PositiveIntValidator())

Initialize an `Item` object with correct values

In [2]:
item = Item(sku="item-sku", quantity=1)
logging.info("item: %s", item)

INFO:root:item: Item(sku='item-sku', quantity=1)


Pass an invalid `sku`

In [30]:
logging.info(vars(Item)["sku"].validate("asdf"))

AttributeError: Field 'sku' must contain a dash ('-') in the middle of the string.

In [3]:
Item(sku="sku", quantity=1)

AttributeError: Field 'sku' must contain a dash ('-') in the middle of the string.

Pass an invalid `quantity`

In [4]:
Item(sku="item-sku", quantity=-1)

AttributeError: quantity must be positive

Now we have a class that validates its inputs and can be easily serialized to a dict.

In [5]:
item = Item(sku="item-sku", quantity=2)
logging.info(asdict(item))

INFO:root:{'sku': 'item-sku', 'quantity': 2}


In [31]:
class DataDescriptor:
    def __get__(self, obj: Any, objtype=None) -> str:
        """Attribute access method.
        
        :param Any obj: Instance of attribute being accessed.
        :param Any objtype: Class of instance being accessed.
        :return: Value stored in private attribute.
        """
        return getattr(obj, '_data')
    
    def __set__(self, obj: Any, value: str) -> None:
        """Attribute assign method.
        
        :param Any obj: Instance of attribute being assigned.
        :param str value: Value being assigned to attribute.
        """
        setattr(obj, '_data', value)

class NonDataDescriptor:
    def __get__(self, obj: Any, objtype=None) -> str:
        """Attribute access method.
        
        :param Any obj: Instance of attribute being accessed.
        :param Any objtype: Class of instance being accessed.
        :return: Value stored in private attribute.
        """
        return "An on demand value"

class User:
    name: str = DataDescriptor()
    email: str = NonDataDescriptor()
    
    
    def __init__(self):
        # non-data descriptor is overriden
        self.email = "my@email.com"
        # data descriptor is called
        self.name = 'Júan Pérez'

In [32]:
user = User()

In [33]:
user.name

'Júan Pérez'